Goal: given an input image $X=[x_1, x_2, ..., x_N]$ with values in $\mathbb{R}$, find a function f(x) so that
 - $f(x_i) \in [0,1] \ \forall i$
 - $\sum_{i=1}^N{f(x_i)} = \mu N$ with $\mu=const$

First idea: $f(x)=sigmoid(x)_c=\frac{1}{1+e^{-x-c}}$ and optimize for $c$.

In [16]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import time

In [12]:
def newton(func, guess, threshold = 1e-7):
    guess = Variable(guess, requires_grad=True)
    value = func(guess)
    i = 0
    while abs(value.item()) > threshold:
        value = func(guess)
        value.backward()
        guess.data -= (value / guess.grad).data
        guess.grad.data.zero_()
        i += 1
    return guess.data, i

In [30]:
def sigmoidNormalize(X, c, min_value=0):
    return torch.sigmoid(X + c)*(1-min_value) + min_value

In [50]:
def getOptimizationFunction(X, target, min_value):
    def f(c):
        X2 = sigmoidNormalize(X, c, min_value)
        current = torch.mean(X2)
        return (current-target)
    return f

In [38]:
X = torch.randn(5, 5) * 10
print("Input:\n", X, sep='')
print("min:", torch.min(X), 'max:', torch.max(X), 'mean:', torch.mean(X))
print()

min_value = 0.1

initialC = torch.tensor(0.0)
print(initialC.shape)

sigX = sigmoidNormalize(X, initialC, min_value)
print("normalized:\n", sigX, sep='')
print("min:", torch.min(sigX), 'max:', torch.max(sigX), 'mean:', torch.mean(sigX))
print()

targetMean = 0.2
print("target mean:", targetMean)

startTime = time.monotonic()
finalC, iteration = newton(getOptimizationFunction(X.cuda(), targetMean, min_value), initialC.cuda())
endTime = time.monotonic()
print("Converged after", iteration, "iterations to an offset of", finalC)
print("Elapsed time:", (endTime-startTime)*1000, "ms")
print()

sigX = sigmoidNormalize(X, finalC, min_value)
print("normalized:\n", sigX, sep='')
print("min:", torch.min(sigX), 'max:', torch.max(sigX), 'mean:', torch.mean(sigX))
print()

Input:
tensor([[ -5.0823,  -7.2611,   8.7220,   8.7264,   0.7389],
        [  1.0328, -17.6282,  -1.7303,   1.4243, -13.0179],
        [  4.8271,  -1.5445,  -0.5326,   3.5634,  -3.4657],
        [  0.2543,  12.4339,  -3.9045,  12.6888, -16.2573],
        [  3.0552,   6.1870,   9.9390,  14.0749,  -8.3066]])
min: tensor(-17.6282) max: tensor(14.0749) mean: tensor(0.3575)

torch.Size([])
normalized:
tensor([[0.1056, 0.1006, 0.9999, 0.9999, 0.7091],
        [0.7637, 0.1000, 0.2355, 0.8254, 0.1000],
        [0.9928, 0.2583, 0.4329, 0.9752, 0.1273],
        [0.6069, 1.0000, 0.1178, 1.0000, 0.1000],
        [0.9595, 0.9982, 1.0000, 1.0000, 0.1002]])
min: tensor(0.1000) max: tensor(1.0000) mean: tensor(0.5843)

target mean: 0.2
Converged after 5 iterations to an offset of tensor(-11.4058, device='cuda:0')
Elapsed time: 16.00000006146729 ms

normalized:
tensor([[0.1000, 0.1000, 0.1575, 0.1578, 0.1000],
        [0.1000, 0.1000, 0.1000, 0.1000, 0.1000],
        [0.1012, 0.1000, 0.1000, 0.1004, 0.

In [48]:
def bisection(func, guess, step=1, threshold=1e-7):
    """
    Assumes func to be monotonic
    """
    
    value = func(guess).item()
    print("guess=%.5f -> value=%.5f"%(guess, value))
    if value > 0:
        right = guess
        right_value = value
        left = guess - step
        left_value = value
        while True:
            value = func(left).item()
            assert value < left_value, "function is not monotonic increasing"
            left_value = value
            print("left=%.5f -> left_value=%.5f"%(left, left_value))
            if left_value < 0:
                break
            step = step * 2
            left = guess - step
    else:
        left = guess
        left_value = value
        right = guess + step
        right_value = value
        while True:
            value = func(right).item()
            assert value > right_value, "function is not monotonic increasing"
            right_value = value
            print("right=%.5f -> right_value=%.5f"%(right, right_value))
            if right_value > 0:
                break
            step = step * 2
            right = guess + step
            
    i = 0
    while abs(value) > threshold:
        # false position method
        guess = (left * right_value - right * left_value) / (right_value - left_value)
        value = func(guess).item()
        print("guess=%.5f -> value=%.5f"%(guess, value))
        # keep left and right value
        if value > 0:
            right = guess
            right_value = value
        else:
            left = guess
            left_value = value
        i += 1
    return guess, i

In [51]:
X = torch.randn(5, 5) * 1
print("Input:\n", X, sep='')
print("min:", torch.min(X), 'max:', torch.max(X), 'mean:', torch.mean(X))
print()

min_value = 0.1

initialC = torch.tensor(-4)
print(initialC.shape)

sigX = sigmoidNormalize(X, initialC, min_value)
print("normalized:\n", sigX, sep='')
print("min:", torch.min(sigX), 'max:', torch.max(sigX), 'mean:', torch.mean(sigX))
print()

targetMean = 0.2
print("target mean:", targetMean)

startTime = time.monotonic()
finalC, iteration = bisection(getOptimizationFunction(X.cuda(), targetMean, min_value), initialC.cuda())
endTime = time.monotonic()
print("Converged after", iteration, "iterations to an offset of", finalC)
print("Elapsed time:", (endTime-startTime)*1000, "ms")
print()

sigX = sigmoidNormalize(X, finalC, min_value)
print("normalized:\n", sigX, sep='')
print("min:", torch.min(sigX), 'max:', torch.max(sigX), 'mean:', torch.mean(sigX))
print()

Input:
tensor([[ 0.9823, -0.6003,  2.1897,  0.6543,  1.5350],
        [ 0.5103,  0.3197,  0.4895, -0.2049,  2.3913],
        [-0.2200,  0.9613,  0.3155, -0.7265, -0.4050],
        [ 2.5826,  0.9836,  1.0097,  1.0198, -0.5601],
        [ 0.6508,  0.2425, -0.4872,  0.5464,  0.2833]])
min: tensor(-0.7265) max: tensor(2.5826) mean: tensor(0.5785)

torch.Size([])
normalized:
tensor([[0.1420, 0.1090, 0.2265, 0.1306, 0.1705],
        [0.1266, 0.1221, 0.1261, 0.1132, 0.2501],
        [0.1130, 0.1411, 0.1220, 0.1079, 0.1109],
        [0.2756, 0.1420, 0.1431, 0.1435, 0.1093],
        [0.1305, 0.1205, 0.1100, 0.1276, 0.1214]])
min: tensor(0.1079) max: tensor(0.2756) mean: tensor(0.1414)

target mean: 0.2
guess=-4.00000 -> value=-0.05859
right=-3.00000 -> right_value=-0.00249
right=-2.00000 -> right_value=0.10328
guess=-3.27610 -> value=-0.02220
guess=-3.05029 -> value=-0.00635
guess=-2.98941 -> value=-0.00166
guess=-2.97379 -> value=-0.00042
guess=-2.96984 -> value=-0.00011
guess=-2.96884 -> valu